In [1]:
# 二つ目のボックスで緯度座標系をWGS84 -> 緯度経度10進数に変換
# 変換後の画像をreprojectに保存

# 三つ目のボックスでreprojectから読み込んだ画像をmergeして
# 各バンドごとに一枚の日本地図を作成する

import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from glob import glob
import os
from tqdm import tqdm
import time
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
list_1 = []

base_path = "D:/LULC/raw_data/1_landsat_new"
seasons = ["winter", "spring", "summer", "autumn"]
# seasons = ["winter", "autumn"]

for season in tqdm(seasons):
    try:
        os.makedirs(base_path + f"/6_landsat8_mosaic/{season}")
    except FileExistsError:
        pass
    
    for i in range(1,8):
        dir_base_path = base_path + f"/4_landsat8_lack_added/{season}"
        band = glob(os.path.join(dir_base_path, "*", f"*band{i}*"))
        

        dst_crs = 'EPSG:4326'

        for img in band:
            scene_name = img.split('\\')[1]
            with rasterio.open(img) as src:
                print(src.crs)
        break
    break

  0%|                                                                  | 0/4 [00:00<?, ?it/s]

EPSG:32655
EPSG:32654
EPSG:32655
EPSG:32655
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32655
EPSG:32655
EPSG:32654
EPSG:32654
EPSG:32655
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32654
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32653
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32653
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652
EPSG:32652


  0%|                                                                  | 0/4 [00:03<?, ?it/s]

EPSG:32652
EPSG:32652
EPSG:32651
EPSG:32651


In [2]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
list_1 = []

base_path = "E:/LULC/raw_data/1_landsat_new"
seasons = ["winter", "spring", "summer", "autumn"]
# seasons = ["winter", "autumn"]

for season in seasons:
    try:
        os.makedirs(base_path + f"/5_landsat8_reproject/{season}")
    except FileExistsError:
        pass

    for i in tqdm(range(1,8)):
        dir_base_path = base_path + f"/4_landsat8_lack_added/{season}"
        band = glob(os.path.join(dir_base_path, "*", f"*band{i}*"))
        

        dst_crs = 'EPSG:4326'

        for img in band:
            scene_name = img.split('\\')[1]
            try:
                os.makedirs(base_path + f"/5_landsat8_reproject/{season}/{scene_name}")
            except FileExistsError:
                pass
            with rasterio.open(img) as src:
                transform, width, height = calculate_default_transform(
                    src.crs, dst_crs, src.width, src.height, *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': dst_crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })
                
#             まったく別の場所の画像も入っていたので例外処理。
                if src.crs == "EPSG:32601" or src.crs == "EPSG:32660":
                    list_1.append(base_path + f"/5_landsat8_reproject/{season}/{scene_name}/band{i}.tif")
                    continue
                    
                try:
                    os.makedirs(base_path + f"/5_landsat8_reproject/{season}/{scene_name}")
                except FileExistsError:
                    pass

            
#  ＝＝＝＝＝＝＝＝以下、保存するコード！＝＝＝＝＝＝＝＝＝＝＝＝＝
#                 from IPython.core.debugger import Pdb; Pdb().set_trace()
                with rasterio.open(f"E:/LULC/raw_data/1_landsat_new/5_landsat8_reproject/{season}/{scene_name}/{scene_name}_band{i}.tif", 'w', **kwargs) as dst:
                    for j in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, j),
                            destination=rasterio.band(dst, j),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=dst_crs,
                            resampling=Resampling.nearest)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [31:26<00:00, 269.47s/it]


In [3]:
for season in tqdm(seasons):
    try:
        os.makedirs(base_path + f"/6_landsat8_mosaic/{season}")
    except FileExistsError:
        pass
    
    for i in range(1,8):
        dir_base_path = base_path + f"/5_landsat8_reproject/{season}"
        outpath = base_path + f"/6_landsat8_mosaic/{season}/band{i}.tif"
        if os.path.exists(outpath):
            print(f"{outpath} is existing!")
            continue

        band = glob(os.path.join(dir_base_path, "*", f"*band{i}.tif"))

        src_files_to_mosiac = []

        for sim in band:
            src = rasterio.open(sim)
            src_files_to_mosiac.append(src)

        print("merge start.")
        start = time.time()
        mos, out_trans = merge(src_files_to_mosiac)
        end = time.time()
        print("merge end.")
        print(f"merge time: {end - start}")

        for sim in src_files_to_mosiac:
            sim.close()

        # show(mos, cmap='terrain', adjust=None)

        with rasterio.open(outpath,"w",driver ='Gtiff',count=1,
                    height= mos.shape[1],
                    width= mos.shape[2],
                    transform= out_trans,
                    crs= src.crs,
                    dtype= src.dtypes[0]
                          ) as dest:
            dest.write(mos)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

merge start.
merge end.
merge time: 94.82027101516724
merge start.
merge end.
merge time: 98.7562575340271
merge start.
merge end.
merge time: 102.46451807022095
merge start.
merge end.
merge time: 104.06537508964539
merge start.
merge end.
merge time: 100.20405769348145
merge start.
merge end.
merge time: 100.55672693252563
merge start.
merge end.
merge time: 100.03838872909546


 25%|████████████████████                                                            | 1/4 [30:30<1:31:30, 1830.05s/it]

merge start.
merge end.
merge time: 91.9767096042633
merge start.
merge end.
merge time: 92.29858303070068
merge start.
merge end.
merge time: 91.2681303024292
merge start.
merge end.
merge time: 90.79126214981079
merge start.
merge end.
merge time: 90.68023443222046
merge start.
merge end.
merge time: 90.4488890171051
merge start.
merge end.
merge time: 90.1705367565155


 50%|████████████████████████████████████████                                        | 2/4 [59:40<1:00:12, 1806.04s/it]

merge start.
merge end.
merge time: 88.1022424697876
merge start.
merge end.
merge time: 88.02093648910522
merge start.
merge end.
merge time: 88.00465440750122
merge start.
merge end.
merge time: 87.95439124107361
merge start.
merge end.
merge time: 87.7407865524292
merge start.
merge end.
merge time: 88.04521584510803
merge start.
merge end.
merge time: 88.11859726905823


 75%|████████████████████████████████████████████████████████████                    | 3/4 [1:28:38<29:45, 1785.82s/it]

merge start.
merge end.
merge time: 91.12065935134888
merge start.
merge end.
merge time: 90.14657092094421
merge start.
merge end.
merge time: 90.46351766586304
merge start.
merge end.
merge time: 90.33564710617065
merge start.
merge end.
merge time: 90.5428409576416
merge start.
merge end.
merge time: 90.43897604942322
merge start.
merge end.
merge time: 91.21223759651184


100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [1:57:56<00:00, 1769.11s/it]


In [ ]:
# num1番目のファイルを親にして，欠損地をnum2番目のファイルで補填する．
num1 = 0

season_ids = [ "0[1-3]", "0[4-6]", "0[7-9]", "1[0-2]" ]

for season_id in season_ids:
    dir_list = glob(f"E:/LULC/raw_data/1_landsat/2_landsat8_open/*20??{season_id}*-SC*")
    dir_base_list = []
#   Get unique id
    for i in range(len(dir_list)):
        dir_base_list.append(dir_list[i].split("\\")[1][4:10])
    scene_list = np.unique(dir_base_list)
#   Get season path
    cnt = 0
    for scene in scene_list:
        season_path = glob(f"E:/LULC/raw_data/1_landsat/2_landsat8_open/LC08{scene}20??{season_id}*-SC*")
        open_paths = season_path
        open_paths.sort()
        
        landsat_paths_pre = glob(open_paths[num1]+"/*band*.tif")
        landsat_paths_pre.sort()
        
        for path in landsat_paths_pre:
            for j in range(1,8):
                with rasterio.open(path) as filename : filename.bounds
                print(filename.crs)
            print("====================")
        print("================================================")
        print("================================================")
        print("================================================")

In [ ]:
mask_path = f"E:/LULC/raw_data/1_landsat/4_landsat8_masked/{season}"

band1 = glob(os.path.join(mask_path, "*", f"*band1*"))
for i in band1:
    with rasterio.open(i) as filename : filename.bounds
    print(filename.meta)